In [1]:
from UTILITY_quickstart import *

In [2]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Base Tao lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [3]:
tao.cmd("show ele PENT")

['Element # 1533',
 'Element Name: PENT',
 'Key: Marker',
 'S_start, S:    994.772815,    994.772815',
 'Ref_time_start, Ref_time:  3.318222E-06,  3.318222E-06',
 '',
 'Attribute values [Only non-zero values shown]:',
 '    1  L                           =  0.0000000E+00 m',
 '    7  OSC_AMPLITUDE               =  0.0000000E+00 m',
 '   24  TILT_CALIB                  =  0.0000000E+00 rad',
 '   25  DE_ETA_MEAS                 =  0.0000000E+00',
 '   53  P0C                         =  1.0000000E+10 eV           BETA                        =  1.0000000E+00',
 '   54  E_TOT                       =  1.0000000E+10 eV           GAMMA                       =  1.9569512E+04',
 '   64  REF_TIME_START              =  3.3182219E-06 sec      50  DELTA_REF_TIME              =  0.0000000E+00 sec',
 '',
 '       TRACKING_METHOD              =  Bmad_Standard             APERTURE_AT                =  Exit_End',
 '       MAT6_CALC_METHOD             =  Auto                      APERTURE_TYPE           

In [4]:
tao.twiss_at_s(ele = "PENT", s_offset = -0.5)

{'beta_a': 0.99991388817197,
 'alpha_a': 0.999686887836314,
 'gamma_a': 1.99954605827816,
 'phi_a': 72.3150579011166,
 'eta_a': 3.37522284376359e-06,
 'etap_a': -7.60864677687215e-06,
 'beta_b': 1.00011621264697,
 'alpha_b': 1.00020486058778,
 'gamma_b': 2.00017731724298,
 'phi_b': 59.622845768874,
 'eta_b': -2.14261248361953e-19,
 'etap_b': -1.31862568529426e-17,
 'c_mat11': 3.12840044307877e-18,
 'c_mat12': -2.23964690538362e-18,
 'c_mat21': 1.68495193627994e-17,
 'c_mat22': -2.39444930010667e-18,
 'gamma_c': 1.0}

In [11]:
def finalFocusSolverObjective(params, tao, ele, s_offset, targetBetaX, targetAlphaX, targetBetaY, targetAlphaY):
    Q5FFkG, Q4FFkG, Q3FFkG, Q2FFkG, Q1FFkG, Q0FFkG  = params
    
    try:
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        setQuadkG(tao, "Q5FF", Q5FFkG)
        setQuadkG(tao, "Q4FF", Q4FFkG)
        setQuadkG(tao, "Q3FF", Q3FFkG)
        setQuadkG(tao, "Q2FF", Q2FFkG)
        setQuadkG(tao, "Q1FF", Q1FFkG)
        setQuadkG(tao, "Q0FF", Q0FFkG)
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")
    
    except: #If Bmad doesn't like the proposed solution, don't crash, give a bad number
        return 1e20

    resultTwiss = tao.twiss_at_s(ele = ele, s_offset = s_offset)
    resultBetaX = resultTwiss["beta_a"]
    resultAlphaX = resultTwiss["alpha_a"]
    resultBetaY = resultTwiss["beta_b"]
    resultAlphaY = resultTwiss["alpha_b"]

    #Golden lattice FF settings as of 2025-01-10-11-17-43
    # Q5FFkG :  -71.837
    # Q4FFkG :  -81.251
    # Q3FFkG :  99.225
    # Q2FFkG :  126.35
    # Q1FFkG :  -235.218
    # Q0FFkG :  126.353
    goldenVector = np.array([ -71.837 , -81.251, 99.225, 126.35, -235.218,  126.353 ])

    #Optionally give a very slight preference to solutions that don't move the magnets very much
    moveError = 1e-6 * np.sum( (params - goldenVector) ** 2 ) 
    
    return (resultBetaX - targetBetaX) ** 2 + (resultAlphaX - targetAlphaX) ** 2 + (resultBetaY - targetBetaY) ** 2 + (resultAlphaY - targetAlphaY) ** 2 + moveError


def finalFocusSolver(
    tao,
    ele = "PENT",
    s_offset = 0.0,
    targetBetaX = 0.5,
    targetAlphaX = 0,
    targetBetaY = 0.5,
    targetAlphaY = 0,
    verbose = False
):
    from scipy.optimize import minimize


    quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"] 
    initialGuess = [getQuadkG(tao, name) for name in quadNameList]

    #For now, just hardcoding bounds... could generalize if required
    #From "bounds.yml" as of 2025-01-10-11-11-35
    # Q5FFkGBounds: (-256, 0)  #BCON = -70
    # Q4FFkGBounds: (-446, 0)  #BCON = -71
    # Q3FFkGBounds: (0, 457)   #BCON = 106
    # Q2FFkGBounds: (0, 167)   #BCON = 112
    # Q1FFkGBounds: (-257, 0)  #BCON = -225
    # Q0FFkGBounds: (0, 167)   #BCON = 112

    bounds = [(-256,0), (-446,0), (0,457), (0,167), (-257,0), (0,167)]


    # Perform optimization using Nelder-Mead
    result = minimize(
        finalFocusSolverObjective, 
        initialGuess, 
        method='Nelder-Mead',
        bounds = bounds,
        args = (tao, ele, s_offset, targetBetaX, targetAlphaX, targetBetaY, targetAlphaY)
    )


    # #Apply best result to the lattice
    # betaSetX, alphaSetX, betaSetY, alphaSetY = result.x
    
    # #Prevent recalculation until changes are made
    # tao.cmd("set global lattice_calc_on = F")
    
    # tao.cmd(f"set element beginning beta_a = {betaSetX}")
    # tao.cmd(f"set element beginning alpha_a = {alphaSetX}")
    # tao.cmd(f"set element beginning beta_b = {betaSetY}")
    # tao.cmd(f"set element beginning alpha_b = {alphaSetY}")
    
    # #Reenable lattice calculations
    # tao.cmd("set global lattice_calc_on = T")

    if verbose:
        print("Optimization Results:")
        print(f"Optimal Parameters: {result.x}")
        print(f"Objective Function Value at Optimal Parameters: {result.fun}")
        print(f"Number of Iterations: {result.nit}")
        print(f"Converged: {result.success}")

    quadVariableNameList = ["Q5FFkG", "Q4FFkG", "Q3FFkG", "Q2FFkG", "Q1FFkG", "Q0FFkG"] 

    
    
    return { quadVariableNameList[i] : result.x[i] for i in range(len(quadVariableNameList)) }


finalFocusSolver(tao, 
                 ele = "PENT",
                 s_offset = -0.25,
                 targetBetaX = 0.5,
                 targetAlphaX = 0.0,
                 targetBetaY = 0.5,
                 targetAlphaY = 0,
                 verbose = True
                 )


Optimization Results:
Optimal Parameters: [ -69.90674642  -80.44269776   98.77831105  129.21861873 -239.74526156
  128.52861982]
Objective Function Value at Optimal Parameters: 3.823099066507106e-05
Number of Iterations: 670
Converged: True


{'Q5FFkG': -69.9067464216445,
 'Q4FFkG': -80.44269775561213,
 'Q3FFkG': 98.77831104715844,
 'Q2FFkG': 129.21861873156104,
 'Q1FFkG': -239.74526155738425,
 'Q0FFkG': 128.5286198150172}

## Confirm we can operate with setLattice()

In [17]:
#Load a default configuration
importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

#Initialize tao object
tao = initializeTao(
    #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    numMacroParticles = 1e4,
    csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

tao.twiss_at_s(ele = "PENT", s_offset = 0.0)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
WARNING! No beam file is specified!
Number of macro particles = 10000.0


{'beta_a': 0.50649204966748,
 'alpha_a': 0.0466683356262582,
 'gamma_a': 1.9786646882376,
 'phi_a': 73.0782192378143,
 'eta_a': -5.53984755575948e-06,
 'etap_a': -1.05998552673305e-05,
 'beta_b': 0.500627846338041,
 'alpha_b': -0.120964597934503,
 'gamma_b': 2.02671993053367,
 'phi_b': 60.5521347355512,
 'eta_b': -2.11809896530596e-18,
 'etap_b': -2.52217362743306e-18,
 'c_mat11': 1.24354101578855e-16,
 'c_mat12': -2.15263057112982e-17,
 'c_mat21': -3.33737707176956e-17,
 'c_mat22': 4.78887427948486e-18,
 'gamma_c': 1.0}

In [18]:
#goldenVector = np.array([ -71.837 , -81.251, 99.225, 126.35, -235.218,  126.353 ])
newMagnetSettings = finalFocusSolver(tao, 
                 ele = "PENT",
                 s_offset = 0.0,
                 targetBetaX = 1.0,
                 targetAlphaX = 0.0,
                 targetBetaY = 1.0,
                 targetAlphaY = 0,
                 verbose = True
                 )

Optimization Results:
Optimal Parameters: [ -67.31617009  -74.97988868  108.98909463  112.05936352 -225.21737042
  114.30355501]
Objective Function Value at Optimal Parameters: 0.0006057083844674432
Number of Iterations: 520
Converged: True


In [19]:
setLattice(tao, **(importedDefaultSettings | newMagnetSettings))

tao.twiss_at_s(ele = "PENT", s_offset = 0.0)

{'beta_a': 0.998944808144639,
 'alpha_a': -2.65814134386144e-05,
 'gamma_a': 1.00105630716865,
 'phi_a': 73.1168481643165,
 'eta_a': -8.07944549818631e-06,
 'etap_a': -7.60471626112937e-06,
 'beta_b': 1.00004044509841,
 'alpha_b': -0.000257527427580277,
 'gamma_b': 0.99995962285502,
 'phi_b': 60.0570456708695,
 'eta_b': -1.77410623306456e-18,
 'etap_b': -2.79993770479687e-18,
 'c_mat11': 1.29653157129085e-16,
 'c_mat12': 1.88716113929479e-17,
 'c_mat21': -1.79870549383777e-17,
 'c_mat22': -2.85914029103542e-18,
 'gamma_c': 1.0}